In [1]:
import category_encoders as ce

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import Featurizer as ft

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Importo dataset

In [2]:
df = pd.read_csv('../data/palabras.csv')
#df_test = pd.read_csv('../data/test.csv')

In [3]:
x = df[df.columns.drop("precio")]
y = df['precio']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

#x_train, x_test, y_train = x, df_test, y

print(f"Original shapes: X={x.shape} y={y.shape}")
print(f"Train shapes: X={x_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={x_test.shape} y={y_test.shape}")

Original shapes: X=(240000, 80) y=(240000,)
Train shapes: X=(192000, 80) y=(192000,)
Test  shapes: X=(48000, 80) y=(48000,)



## Completo los datos faltantes

In [4]:
def getDropCols():
    return ['titulo', 'descripcion', 'direccion', 'id', 'dia', 'mes'] # temporal

def getOneHotCols():
    return ['tipodepropiedad', 'provincia']

def getBinaryCols():
    return ['ciudad', 'idzona']


import category_encoders as ce

OHE = ce.OneHotEncoder(cols=getOneHotCols(), handle_unknown='ignore', use_cat_names=True)
BE = ce.BinaryEncoder(cols=getBinaryCols(), handle_unknown='ignore')

In [5]:
X_train = ft.main(x_train, OHE, BE, 'train')
X_test = ft.main(x_test, OHE, BE, 'test')

NameError: name 'imputar' is not defined

In [27]:
oneHot = ['tipodepropiedad','provincia']
binary = ['ciudad','ciudad-provincia']
all_cats = ['tipodepropiedad','provincia','ciudad','ciudad-provincia']

 ## Encode


In [28]:
x_train = ft.OHE(x_train,oneHot)
x_test = ft.OHE(x_test,oneHot)

In [29]:
x_train = ft.label(x_train,binary)
x_test = ft.label(x_test,binary)

ValueError: bad input shape (192000, 2)

In [ ]:
df_ids = x_test["id"]
x_train = x_train.drop(columns=["id"])
x_test = x_test.drop(columns=["id"])

In [ ]:
for col in x_train.columns[~x_train.columns.isin(x_test.columns)]:
    x_test[col] = 0

for col in x_test.columns[~x_test.columns.isin(x_train.columns)]:
    x_train[col] = 0

    
print(x_test.shape[1] == x_train.shape[1])

print(f"Train shapes: X={x_train.shape} y={x_train.shape}")
print(f"Test  shapes: X={x_test.shape}  y={x_test.shape}")

In [ ]:
x_train = x_train[x_test.columns]


## Modelo y entreno

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=100,n_jobs=-1,objective="reg:squarederror",eval_metric="rmse")

In [ ]:
model.fit(x_train, y_train)

## Predicciones

In [ ]:
ft.predecir(model,x_train,y_train,x_test,y_test)

In [ ]:
plt.rcParams["figure.figsize"] = [15, 15]

importancia = model.feature_importances_

d = {'x':x_train.columns,'y':importancia}

importancia = pd.DataFrame(d)
importancia = importancia[importancia.y>0.02]
ax = plt.barh(importancia.x,importancia.y)

## Exportar tests

In [15]:
#Predigo los precios
predict = model.predict(x_test)

In [16]:
result = df_ids.astype("int32").to_frame()
result["target"] = predict.astype("int32")

In [17]:
result = df_ids.astype("int32").to_frame()
result["target"] = predict.astype("int32")
result.to_csv("../data/results/resultXGB_Features.csv", index=False)

## Tunning

In [18]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
max_depth = [5,10]
gamma = [0,5,15, 31, 50]
colsample_bytree = [0.2, 0.5, 1, 0]
min_child_weight = [5,1,10]



random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'gamma': gamma,
               'colsample_bytree': colsample_bytree,
               'min_child_weight': min_child_weight
              }

In [19]:
from sklearn.model_selection import RandomizedSearchCV

from xgboost import XGBRegressor

model = XGBRegressor()
random_search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25,
                               n_jobs = 2,
                               verbose=2)

random_search.fit(x_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits


/home/peter/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: 

In [ ]:
rf_random = random_search.best_estimator_

In [ ]:
rf_random